In [1]:
!cat ../README.md

# Super Resolution, On The Edge, For The Win!
*by Akshay Chawla and Priyam Tejaswin*

Main reference paper : [Real-Time Video Super-Resolution with Spatio-Temporal Networks and Motion Compensation, CVPR 2017](http://openaccess.thecvf.com/content_cvpr_2017/papers/Caballero_Real-Time_Video_Super-Resolution_CVPR_2017_paper.pdf)
```
.
├── README.md
├── papers ## PDFs for review and reference (not committed).
├── src_tensorflow ## Code in tf (using `tensorflow.keras`). 
└── src_tf2 ## Code in tf2_alpha.
```

## Dependencies
- `src_tensorflow` uses **tensorflow 1.13.1**. This is the latest stable release as of creating this repository.
- `src_tf2` uses **tensorflow 2.0** which is still an alpha build.

## System Design
![](system_blocks.jpg)

## References
- VESPCN implementation (unofficial, TF) - <https://github.com/LoSealL/VideoSuperResolution/blob/master/VSR/Models/Vespcn.py>
- Fully-featured TF2 implementation of YOLOv3 (for checking TF2 oddities) - <https://github.com/zzh8829/yolov3-tf

In [2]:
import tensorflow as tf

In [3]:
tf.enable_eager_execution()
print tf.executing_eagerly()

True


In [85]:
dataset = tf.data.TextLineDataset("../README.md")
dataset = dataset.map(lambda line: tf.string_split([line]).values)
dataset = dataset.filter(lambda row: tf.size(row) > 3)
dataset = dataset.map(lambda row: tf.slice(row, [0], [2]))
dataset = dataset.window(size=3, shift=1, drop_remainder=True) ## THIS WILL RETURN A DATASET!!!!
dataset = dataset.flat_map(lambda dset: dset.batch(3))
## How to shuffle?

In [89]:
dataset.shuffle?

In [86]:
for i in range(5):
    for batch in dataset:
        print i, batch

0 tf.Tensor(
[['#' 'Super']
 ['*by' 'Akshay']
 ['Main' 'reference']], shape=(3, 2), dtype=string)
0 tf.Tensor(
[['*by' 'Akshay']
 ['Main' 'reference']
 ['\xe2\x94\x9c\xe2\x94\x80\xe2\x94\x80' 'papers']], shape=(3, 2), dtype=string)
0 tf.Tensor(
[['Main' 'reference']
 ['\xe2\x94\x9c\xe2\x94\x80\xe2\x94\x80' 'papers']
 ['\xe2\x94\x9c\xe2\x94\x80\xe2\x94\x80' 'src_tensorflow']], shape=(3, 2), dtype=string)
0 tf.Tensor(
[['\xe2\x94\x9c\xe2\x94\x80\xe2\x94\x80' 'papers']
 ['\xe2\x94\x9c\xe2\x94\x80\xe2\x94\x80' 'src_tensorflow']
 ['\xe2\x94\x94\xe2\x94\x80\xe2\x94\x80' 'src_tf2']], shape=(3, 2), dtype=string)
0 tf.Tensor(
[['\xe2\x94\x9c\xe2\x94\x80\xe2\x94\x80' 'src_tensorflow']
 ['\xe2\x94\x94\xe2\x94\x80\xe2\x94\x80' 'src_tf2']
 ['-' '`src_tensorflow`']], shape=(3, 2), dtype=string)
0 tf.Tensor(
[['\xe2\x94\x94\xe2\x94\x80\xe2\x94\x80' 'src_tf2']
 ['-' '`src_tensorflow`']
 ['-' '`src_tf2`']], shape=(3, 2), dtype=string)
0 tf.Tensor(
[['-' '`src_tensorflow`']
 ['-' '`src_tf2`']
 ['-' 'VES